In [ ]:
import conducto as co
import commands

In [ ]:
path = "/conducto/data/pipeline"
root = co.Serial(image=get_image())
root

In [ ]:
# Get data from keras for testing and training
root["Get Data"] = co.Exec(commands.get_data, f"{path}/raw")

# Split it into train/test/validate
root["Split"] = co.Exec(
    commands.split_data,
    input=f"{path}/raw",
    train=f"{path}/train",
    test=f"{path}/test",
    validate=f"{path}/validate",
    split=[60,20,20]
)
root

In [ ]:
root["Models"] = co.Parallel()
for model in "pytorch", "keras", "sklearn":
    model_node = co.Serial()
    model_node["Fit"] = co.Exec(commands.fit, data=f"{path}/train", model=f"{path}/{model}")
    model_node["Test"] = co.Exec(commands.test, data=f"{path}/test", model=f"{path}/{model}", result=f"{path}/result/{model}")
    root["Models"][model] = model_node

root["Summary"] = co.Exec(commands.summarize, results=f"{path}/result")
root

In [ ]:
root.launch(local=True)
